# Example 14.5 Pressure Relief

A 2000 liter vessel contains 1000 kg of a 10 wt% water and 90 wt% propanediol-1,2 mixture, stored at its own vapor pressure, and protected by a rupture disk which is set to activate at 3 bar. A heat flux of 1 MW is applied to the fluid, which will cause a relief event. Determine the mass flow rate which will exit the rupture disk.


Use the NRTL model with A12 = 0.1078, A21=-.2811, B12=62.0818, B21=-1.4101, alpha12=0.3, and an ideal gas model for the vapor phase.

In [1]:
from thermo import ChemicalConstantsPackage, NRTL, IdealGas, GibbsExcessLiquid, FlashVL
from chemicals import ws_to_zs, mixing_simple, rho_to_Vm
from scipy.constants import bar
# Propylene glycol is another more common synonym than propanediol-1,2
constants, correlations = ChemicalConstantsPackage.from_IDs(['water', 'Propylene glycol'])
from scipy.constants import calorie, R
zs = ws_to_zs(ws=[.1, .9], MWs=constants.MWs)


P = 1*bar
T = 300 # Initial guess
V_vessel = 2.0 # m^2
m = 1000 # kg in the tank

# Calculate the average molecular weight
MW_avg = mixing_simple(zs, constants.MWs) # g/mol
# Compute the overall density for the contents of the tank
bulk_density = m/V_vessel # kg/m^3
# Compute the molar volume of the bulk solution
Vm_avg = rho_to_Vm(rho=bulk_density, MW=MW_avg) # m^3/mol
moles = V_vessel/Vm_avg # Compute the total number of moles of substance in the tank
moles

17378.254767083745

In [2]:
# Configure the liquid phase
# As this system is low-pressure, the Poynting correction factor is ignored
# Since the gas phase is set to the idea gas, it makes no sense to use
# Phi correction factors for gas-nonideality in the liquid phase either
tau_as = [[0, 0.1078/R*calorie], [-.2811/R*calorie, 0]]
tau_bs = [[0, 62.0818/R*calorie], [-1.4101/R*calorie, 0]]
alphaC =  [[0, 0.3],[.3, 0]]
GE = NRTL(T=T, xs=zs, tau_as=tau_as, tau_bs=tau_bs)

liquid = GibbsExcessLiquid(VaporPressures=correlations.VaporPressures,
                           HeatCapacityGases=correlations.HeatCapacityGases,
                           VolumeLiquids=correlations.VolumeLiquids,
                           GibbsExcessModel=GE,
                           equilibrium_basis='Psat', caloric_basis='Psat',
                           T=T, P=P, zs=zs)

In [3]:
# Configure the ideal gas
gas = IdealGas(T=T, P=P, zs=zs, HeatCapacityGases=correlations.HeatCapacityGases)

In [4]:
from scipy.optimize import brenth

flasher = FlashVL(constants, correlations, liquid=liquid, gas=gas)
# To find the temperature at which the rupture disk will break open, we have two specs
# 3 bar and the molar volume of the bulk solution calculated earlier, `Vm_avg`.
# Iteration on PT flashes is one solution.

def V_error(T):
    PT = flasher.flash(T=T, P=P, zs=zs)
    return PT.V() - Vm_avg

T_release = brenth(V_error, 300, 600, xtol=1e-6)
T_release

405.05615108404197

In [5]:
T_release-273.15

131.906151084042

In [6]:
[i.method for i in correlations.VolumeLiquids]

['HEOS_FIT', 'DIPPR_PERRY_8E']